In [1]:
# Loading packages
import re
import pandas as pd
import numpy as np
import ast

import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/franciscorfafonso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#load cleaned dataset
df = pd.read_excel('Tweets.xlsx', sheet_name='tweets_wipdata', engine='openpyxl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58447 entries, 0 to 58446
Data columns (total 35 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             58447 non-null  int64  
 1   text                                 58447 non-null  object 
 2   author_id                            58447 non-null  int64  
 3   created_at                           58447 non-null  object 
 4   campaign_week                        58447 non-null  int64  
 5   public_metrics.retweet_count         58447 non-null  int64  
 6   public_metrics.reply_count           58447 non-null  int64  
 7   public_metrics.like_count            58447 non-null  int64  
 8   public_metrics.quote_count           58447 non-null  int64  
 9   user.id                              58447 non-null  int64  
 10  user.username                        58447 non-null  object 
 11  user.created_at             

In [3]:
# Define a function to convert a string to a list
def str_to_list(s):
    if isinstance(s, str):
        return ast.literal_eval(s)
    else:
        return []

df['check_city'] = df['check_city'].apply(str_to_list)

In [4]:
#ignore list based on past iteractions
ignore_list = ['home','freedom',
               'hometown','cook',
               'media','strong',
               'homeland',
               'central',
               'parade',
               'hope',
               'advance',
               'agency',
               'agenda',
               'ages',
               'job',
               'victory',
               'alert',
               'champion',
               'alex',
               'jones',
               'alexander',
               'national',
               'aline',
               'america',
               'and',
               'anthony',
               'arsenal',
               'art',
               'ashley',
               'ball',
               'banks',
               'banning',
               'barnes',
               'bear',
               'beautiful',
               'between',
               'black',
               'branch',
               'joe',
               'brave',
               'independence',
               'target',
               'bucks',
               'bunch',
               'call',
               'camp',
               'carbon',
               'carson',
               'carlton',
               'case',
               'cash',
               'celebration',
               'safe',
               'center',
               'liberal',
               'fellowship',
               'century',
               'challenge',
               'chambers',
               'chance',
               'charles',
               'council',
               'chase',
               'time',
               'energy',
               'china',
               'church',
               'clear',
               'coal',
               'clinton',
               'colbert',
               'cole',
               'tom',
               'college',
               'ruby',
               'hall',
               'commerce',
               'nelson',
               'confidence',
               'confederate',
               'congress',
               'cope',
               'corn',
               'cov',
               'ever',
               'many',
               'wayne',
               'craig',
               'cross',
               'curry',
               'cycle',
               'daniel',
               'david',
               'day',
               'russia',
               'democrat',
               'friendship',
               'economy',
               'equality',
               'good',
               'farmers',
               'federal',
               'felt',
               'fine',
               'fly',
               'force',
               'likely',
               'may',
               'safe',
               'united',
               'gas',
               'gates',
               'globe',
               'good',
               'lucky',
               'strong',
               'grant',
               'green',
               'happy',
               'high',
               'hollywood',
               'hope',
               'indian',
               'industry',
               'international',
               'issue',
               'jackson',
               'james',
               'jerusalem',
               'jet',
               'job',
               'joe',
               'joy',
               'justice',
               'tony',
               'lack',
               'large',
               'laws',
               'lead',
               'liberal',
               'liberty',
               'lincoln',
               'little',
               'luck',
               'start',
               'market',
               'martin',
               'may',
               'max',
               'mission',
               'protection',
               'moran',
               'moscow',
               'national',
               'nice',
               'normal',
               'relief',
               'radical',
               'oil',
               'price',
               'progress',
               'prosperity',
               'public',
               'quality',
               'reform',
               'republic',
               'republican',
               'roe',
               'roll',
               'ronald',
               'rush',
               'ryan',
               'russia',
               'scott',
               'section',
               'security',
               'sells',
               'selling',
               'shannon',
               'sharon',
               'speed',
               'start',
               'stop',
               'supply',
               'taylor',
               'tell',
               'thomas',
               'trail',
               'union',
               'united',
               'universal',
               'university',
               'victory',
               'wade',
               'walker',
               'wall',
               'walters',
               'elizabeth',
               'warren',
               'welcome',
               'white',
               'worth',
               'young'
               ]


def check_difference(row):
    # Get the lists from 'check_city' and 'gpe_city' columns
    check_city_list = row['check_city']

    # Calculate the difference between the two lists
    difference = list(set(check_city_list) - set(ignore_list))

    return difference

# Apply the function to each row in the DataFrame
df['check_city_filtered'] = df[df['check_city'].str.len() > 0].apply(check_difference, axis=1)
df.head(20)

,tweet_id,text,author_id,created_at,campaign_week,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user.id,...,result_pctg,result_votes,position,total_votes_casted,state_short,winner_margin_for_runners_up,check_city,gpe_city,difference,check_city_filtered
0,1589769441016877056,Deeply honored to be standing in the historic ...,913415985591439362,2022-11-07,36,37,18,144,2,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,"[new hampshire, exeter]",['new hampshire'],['exeter'],"[exeter, new hampshire]"
1,1589758870242500608,Full house tonight! Looking forward to having ...,913415985591439362,2022-11-07,36,73,42,270,1,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,"[new hampshire, exeter]",['new hampshire'],['exeter'],"[exeter, new hampshire]"
2,1589753955592331264,Everyone’s here for the final town hall in Exe...,913415985591439362,2022-11-07,36,99,23,569,4,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[exeter],[],['exeter'],[exeter]
3,1589723041205886976,Career politicians like @SenatorHassan will sa...,913415985591439362,2022-11-07,36,81,51,232,9,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[],[],[],NaN
4,1589684478540406784,@sixohfree Thank you!!! 🇺🇸🇺🇸,913415985591439362,2022-11-07,36,0,0,2,0,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[],[],[],NaN
5,1589682964933197824,LET'S GO! https://t.co/dpCB6RFCZC,913415985591439362,2022-11-07,36,65,24,210,0,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[],[],[],NaN
6,1589682136436465664,🚨🚨🚨Join @GovChrisSununu in supporting our Repu...,913415985591439362,2022-11-07,36,65,17,200,1,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,"[new hampshire, washington]","['new hampshire', 'washington']",[],"[new hampshire, washington]"
7,1589673963508363265,🚨—ENDORSEMENT ALERT: Thank you to the @LogCabi...,913415985591439362,2022-11-07,36,60,15,152,4,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[],[],[],NaN
8,1589668789863342080,.@newsmax @Tom_Basile covering our Seabrook me...,913415985591439362,2022-11-07,36,34,10,96,1,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,"[new hampshire, washington, seabrook]",['washington'],"['new hampshire', 'seabrook']","[seabrook, new hampshire, washington]"
9,1589663927956549632,Started the morning off right! Linda’s Breakfa...,913415985591439362,2022-11-07,36,55,12,186,5,913415985591439362,...,0.444346,275928,2,620975,NH,0.090608,[seabrook],[],['seabrook'],[seabrook]


In [5]:
def replace_values(row):
    col1 = row['process_text_check']
    list_to_check = row['check_city_filtered']

    # Check if the row is iterable
    if isinstance(list_to_check, list):
        for val in list_to_check:
            if val in col1:
                col1 = col1.replace(val,'local_tweet')

    return col1

df['PreProcessedText_LocalTweet'] = df.apply(replace_values, axis=1)
df['PreProcessedText_WithoutCity'] = df['PreProcessedText_LocalTweet'].apply(lambda x: x.replace('local_tweet','') if isinstance(x, str) and 'local_tweet' in x else x)
df.head()

,tweet_id,text,author_id,created_at,campaign_week,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user.id,...,position,total_votes_casted,state_short,winner_margin_for_runners_up,check_city,gpe_city,difference,check_city_filtered,PreProcessedText_LocalTweet,PreProcessedText_WithoutCity
0,1589769441016877056,Deeply honored to be standing in the historic ...,913415985591439362,2022-11-07,36,37,18,144,2,913415985591439362,...,2,620975,NH,0.090608,"[new hampshire, exeter]",['new hampshire'],['exeter'],"[exeter, new hampshire]",deeply honored to be standing in the historic ...,deeply honored to be standing in the historic ...
1,1589758870242500608,Full house tonight! Looking forward to having ...,913415985591439362,2022-11-07,36,73,42,270,1,913415985591439362,...,2,620975,NH,0.090608,"[new hampshire, exeter]",['new hampshire'],['exeter'],"[exeter, new hampshire]",full house tonight looking forward to having w...,full house tonight looking forward to having w...
2,1589753955592331264,Everyone’s here for the final town hall in Exe...,913415985591439362,2022-11-07,36,99,23,569,4,913415985591439362,...,2,620975,NH,0.090608,[exeter],[],['exeter'],[exeter],everyone s here for the final town hall in loc...,everyone s here for the final town hall in be...
3,1589723041205886976,Career politicians like @SenatorHassan will sa...,913415985591439362,2022-11-07,36,81,51,232,9,913415985591439362,...,2,620975,NH,0.090608,[],[],[],NaN,career politicians like will say anything to g...,career politicians like will say anything to g...
4,1589684478540406784,@sixohfree Thank you!!! 🇺🇸🇺🇸,913415985591439362,2022-11-07,36,0,0,2,0,913415985591439362,...,2,620975,NH,0.090608,[],[],[],NaN,thank you,thank you


In [7]:
#considere only tweets written in english
df = df[df['language_check'] == 'en']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57910 entries, 0 to 58446
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             57910 non-null  int64  
 1   text                                 57910 non-null  object 
 2   author_id                            57910 non-null  int64  
 3   created_at                           57910 non-null  object 
 4   campaign_week                        57910 non-null  int64  
 5   public_metrics.retweet_count         57910 non-null  int64  
 6   public_metrics.reply_count           57910 non-null  int64  
 7   public_metrics.like_count            57910 non-null  int64  
 8   public_metrics.quote_count           57910 non-null  int64  
 9   user.id                              57910 non-null  int64  
 10  user.username                        57910 non-null  object 
 11  user.created_at             

In [8]:
# Tokenize words
def tokenize_words(words):
    if (type(words) != str) or (word_tokenize(words) == ''):
        return np.nan
    else:
        tt = TweetTokenizer()
        return tt.tokenize(words)

In [9]:
# Function to create text from words
def recreateText(words):
    if type(words) == list:
        temp_str = (' ').join(words)
        return temp_str
    else:
        return np.nan

In [10]:
# Function to remove stop words
def removeStopWords(t, stop_words):
    if type(t) == list:
        return [w for w in t if not w in stop_words]
    else:
        return np.nan

In [11]:
# Function to lemmatize words
def lemmatize(words):
    if type(words) == list:
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(x) for x in words]
    else:
        return np.nan

In [12]:
# Tokekinze words
df['Words'] =  df['PreProcessedText_WithoutCity'].apply(tokenize_words)

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['WordsCleaned'] = df['Words'].apply(removeStopWords,stop_words=stop_words)

# Remove all reviews with no words
df = df[df['WordsCleaned'].str.len()>0]

# Lemmative words
df['WordsLemmatized'] = df['WordsCleaned'].apply(lemmatize)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57790 entries, 0 to 58446
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   tweet_id                             57790 non-null  int64  
 1   text                                 57790 non-null  object 
 2   author_id                            57790 non-null  int64  
 3   created_at                           57790 non-null  object 
 4   campaign_week                        57790 non-null  int64  
 5   public_metrics.retweet_count         57790 non-null  int64  
 6   public_metrics.reply_count           57790 non-null  int64  
 7   public_metrics.like_count            57790 non-null  int64  
 8   public_metrics.quote_count           57790 non-null  int64  
 9   user.id                              57790 non-null  int64  
 10  user.username                        57790 non-null  object 
 11  user.created_at             

In [14]:
#remove unnecessary columns for further modelling and analysis
df.drop(columns=['author_id', 'public_metrics.retweet_count','public_metrics.reply_count','public_metrics.like_count','public_metrics.quote_count',
                 'user.id','user.username','user.created_at','user.public_metrics.followers_count','user.public_metrics.following_count',
                 'user.public_metrics.tweet_count','user.public_metrics.listed_count','Length','language','language_check','PreProcessedText',
                 'state_short','check_city','gpe_city','difference','PreProcessedText_LocalTweet'], inplace=True)

In [15]:
# Save the DataFrame to a pickle file
df.to_pickle('data_preprocessed_filtered.pkl')